In [1]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [3]:
train = pd.read_csv("../../../data/real_or_not_nlp/train.csv")

In [4]:
train.head()

id keyword location                                               text  \
0   1     NaN      NaN  Our Deeds are the Reason of this #earthquake M...   
1   4     NaN      NaN             Forest fire near La Ronge Sask. Canada   
2   5     NaN      NaN  All residents asked to 'shelter in place' are ...   
3   6     NaN      NaN  13,000 people receive #wildfires evacuation or...   
4   7     NaN      NaN  Just got sent this photo from Ruby #Alaska as ...   

   target  
0       1  
1       1  
2       1  
3       1  
4       1

In [5]:
train.describe()

id      target
count   7613.000000  7613.00000
mean    5441.934848     0.42966
std     3137.116090     0.49506
min        1.000000     0.00000
25%     2734.000000     0.00000
50%     5408.000000     0.00000
75%     8146.000000     1.00000
max    10873.000000     1.00000

In [6]:
train.keyword.unique()

array([nan, 'ablaze', 'accident', 'aftershock', 'airplane%20accident',
       'ambulance', 'annihilated', 'annihilation', 'apocalypse',
       'armageddon', 'army', 'arson', 'arsonist', 'attack', 'attacked',
       'avalanche', 'battle', 'bioterror', 'bioterrorism', 'blaze',
       'blazing', 'bleeding', 'blew%20up', 'blight', 'blizzard', 'blood',
       'bloody', 'blown%20up', 'body%20bag', 'body%20bagging',
       'body%20bags', 'bomb', 'bombed', 'bombing', 'bridge%20collapse',
       'buildings%20burning', 'buildings%20on%20fire', 'burned',
       'burning', 'burning%20buildings', 'bush%20fires', 'casualties',
       'casualty', 'catastrophe', 'catastrophic', 'chemical%20emergency',
       'cliff%20fall', 'collapse', 'collapsed', 'collide', 'collided',
       'collision', 'crash', 'crashed', 'crush', 'crushed', 'curfew',
       'cyclone', 'damage', 'danger', 'dead', 'death', 'deaths', 'debris',
       'deluge', 'deluged', 'demolish', 'demolished', 'demolition',
       'derail', 'der

In [7]:
from collections import Counter

In [8]:
Counter(train.keyword)

Counter({nan: 61,
         'ablaze': 36,
         'accident': 35,
         'aftershock': 34,
         'airplane%20accident': 35,
         'ambulance': 38,
         'annihilated': 34,
         'annihilation': 29,
         'apocalypse': 32,
         'armageddon': 42,
         'army': 34,
         'arson': 32,
         'arsonist': 34,
         'attack': 36,
         'attacked': 35,
         'avalanche': 30,
         'battle': 26,
         'bioterror': 37,
         'bioterrorism': 30,
         'blaze': 38,
         'blazing': 34,
         'bleeding': 35,
         'blew%20up': 33,
         'blight': 32,
         'blizzard': 37,
         'blood': 35,
         'bloody': 35,
         'blown%20up': 33,
         'body%20bag': 33,
         'body%20bagging': 33,
         'body%20bags': 41,
         'bomb': 34,
         'bombed': 38,
         'bombing': 29,
         'bridge%20collapse': 35,
         'buildings%20burning': 35,
         'buildings%20on%20fire': 33,
         'burned': 33,
         'bu

In [9]:
train.location.unique()

array([nan, 'Birmingham', 'Est. September 2012 - Bristol', ...,
       'Vancouver, Canada', 'London ', 'Lincoln'], dtype=object)

In [10]:
Counter(train.target)

Counter({1: 3271, 0: 4342})

In [11]:
train.text

0       Our Deeds are the Reason of this #earthquake M...
1                  Forest fire near La Ronge Sask. Canada
2       All residents asked to 'shelter in place' are ...
3       13,000 people receive #wildfires evacuation or...
4       Just got sent this photo from Ruby #Alaska as ...
                              ...                        
7608    Two giant cranes holding a bridge collapse int...
7609    @aria_ahrary @TheTawniest The out of control w...
7610    M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...
7611    Police investigating after an e-bike collided ...
7612    The Latest: More Homes Razed by Northern Calif...
Name: text, Length: 7613, dtype: object

In [12]:
import re
from string import punctuation
from nltk.corpus import stopwords

from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer

english_stopwords = stopwords.words('english')
stemmer = PorterStemmer()

def transform_tweet(text):
    text = re.sub("#", "", text)
    text = re.sub("\@[a-zA-Z_-]+", "", text)
    text = re.sub("^\s+", "", text)
    text = re.sub("\s+$", "", text)
    text = re.sub(f'[{punctuation}]', "", text)
    
    tokenizer = TweetTokenizer()
    words = tokenizer.tokenize(text)
    words = [word for word in words if word not in english_stopwords]
    words = [stemmer.stem(word) for word in words]
    
    return words

In [13]:
transform_tweet(train.iloc[10].text)

['three', 'peopl', 'die', 'heat', 'wave', 'far']

In [14]:
transformed_text = [transform_tweet(text) for text in train.text]

In [15]:
from tqdm import tqdm

In [16]:
freqs = {}
for index, sentence in tqdm(train.iterrows()):
    for word in transformed_text[index]:
        if sentence.target == 1:
            freqs[(word, 1.0)] = freqs.get((word, 1.0), 0) + 1
        else:
            freqs[(word, 0.0)] = freqs.get((word, 1.0), 0) + 1

7613it [00:01, 3885.98it/s]


In [17]:
freqs

{('our', 1.0): 16,
 ('deed', 1.0): 1,
 ('reason', 1.0): 8,
 ('earthquak', 1.0): 47,
 ('may', 1.0): 50,
 ('allah', 1.0): 6,
 ('forgiv', 1.0): 1,
 ('us', 1.0): 14,
 ('forest', 1.0): 50,
 ('fire', 1.0): 270,
 ('near', 1.0): 49,
 ('La', 1.0): 11,
 ('rong', 1.0): 1,
 ('sask', 1.0): 1,
 ('canada', 1.0): 10,
 ('all', 1.0): 22,
 ('resid', 1.0): 9,
 ('ask', 1.0): 5,
 ('shelter', 1.0): 6,
 ('place', 1.0): 17,
 ('notifi', 1.0): 1,
 ('offic', 1.0): 41,
 ('No', 1.0): 34,
 ('evacu', 1.0): 101,
 ('order', 1.0): 24,
 ('expect', 1.0): 22,
 ('13000', 1.0): 4,
 ('peopl', 1.0): 106,
 ('receiv', 1.0): 4,
 ('wildfir', 1.0): 80,
 ('california', 1.0): 115,
 ('just', 1.0): 19,
 ('got', 1.0): 29,
 ('sent', 1.0): 4,
 ('photo', 1.0): 28,
 ('rubi', 1.0): 1,
 ('alaska', 1.0): 5,
 ('smoke', 1.0): 14,
 ('pour', 1.0): 2,
 ('school', 1.0): 32,
 ('rockyfir', 1.0): 4,
 ('updat', 1.0): 36,
 ('hwi', 1.0): 10,
 ('20', 1.0): 14,
 ('close', 1.0): 29,
 ('direct', 1.0): 6,
 ('due', 1.0): 24,
 ('lake', 1.0): 9,
 ('counti', 1.0):

In [18]:
# EVALUATION

In [19]:
test = pd.read_csv("../../../data/real_or_not_nlp/test.csv")

In [20]:
test.head()

id keyword location                                               text
0   0     NaN      NaN                 Just happened a terrible car crash
1   2     NaN      NaN  Heard about #earthquake is different cities, s...
2   3     NaN      NaN  there is a forest fire at spot pond, geese are...
3   9     NaN      NaN           Apocalypse lighting. #Spokane #wildfires
4  11     NaN      NaN      Typhoon Soudelor kills 28 in China and Taiwan

In [21]:
test.describe()

id
count   3263.000000
mean    5427.152927
std     3146.427221
min        0.000000
25%     2683.000000
50%     5500.000000
75%     8176.000000
max    10875.000000

In [22]:
# for _, row in test.iterrows():
#     transformed_tweet = transform_tweet(row.text)
#     print(transformed_tweet)

In [23]:
logprior = np.log(sum(train.target == 1)) - np.log(sum(train.target == 0))
V = len(train)
N_pos = sum(train.target == 1)
N_neg = sum(train.target == 0)

In [24]:
print(logprior)
print(V)
print(N_pos)
print(N_neg)

-0.28323932289985443
7613
3271
4342


In [25]:
def evaluate_tweet(transformed_tweet):
    result = logprior
    
    for word in transformed_tweet:
        freq_pos = freqs.get((word, 1.0), 0.0)
        freq_neg = freqs.get((word, 0.0), 0.0)
        
        result += (np.log((freq_pos + 1.0) / float(N_pos + V)) - np.log((freq_neg + 1.0) / float(N_neg + V)))
        
    return result

In [26]:
#evaluate_tweet(transform_tweet(test.iloc[2]['text']))
results = []
for index, tweet in tqdm(test.iterrows()):
    print(tweet.text)
    result = evaluate_tweet(transform_tweet(tweet.text))
    print(result)
    results.append(int(result >= 0))
    
test['target'] = results

176it [00:00, 754.19it/s]

Just happened a terrible car crash
0.20508546118490756
Heard about #earthquake is different cities, stay safe everyone.
0.1524611543602763
there is a forest fire at spot pond, geese are fleeing across the street, I cannot save them all
3.70929625257574
Apocalypse lighting. #Spokane #wildfires
0.8935533806385108
Typhoon Soudelor kills 28 in China and Taiwan
3.5172427461571463
We're shaking...It's an earthquake
0.08533939405764013
They'd probably still show more life than Arsenal did yesterday, eh? EH?
-0.012019979800116154
Hey! How are you?
-0.35257286591116976
What a nice hat?
-0.5902753077406651
Fuck off!
-0.12039067142361404
No I don't like cold!
0.09965696607298336
NOOOOOOOOO! Don't do that!
-0.11494584877837877
No don't tell me that!
0.10027078821517676
What if?!
-0.2183710797838181
Awesome!
-0.5948486510187312
Birmingham Wholesale Market is ablaze BBC News - Fire breaks out at Birmingham's Wholesale Market http://t.co/irWqCEZWEU
1.1880857563936265
@sunkxssedharry will you wear sho

383it [00:00, 876.33it/s]


RT @TheAdvocateMag: Owner of Chicago-Area Gay Bar Admits to Arson Scheme http://t.co/wHTMwtgROJ #p2 #LGBT
-1.6022010001811626
#LGBTQ News ?? Owner of Chicago-Area Gay Bar Admits to Arson Scheme: Frank Elliott pleaded... http://t.co/sGb9vNWqUx Via @TheAdvocateMag
-1.61686671518844
Mourning notices for stabbing arson victims stir Û÷politics of griefÛª in Israel: Posters for Shira Banki and A... http://t.co/6o92wDfcLu
1.010708171104024
Contra Costa Co. authorities arrest suspected serial arsonist: Authorities believe thatÛ_ http://t.co/bzCmzM7bi5 | http://t.co/LBQldyKgdp
0.4390418781855816
Owner of Chicago-Area Gay Bar Admits to Arson Scheme: Frank Elliott pleaded guilty to hiring an arsonist to to... http://t.co/L82mrYxfNK
-2.592985482777163
@local_arsonist @Cloudy_goldrush Man what ???? they be on some other shit
-0.14431792709070912
mo the way she says 'carry' https://t.co/vQzRUTHRNU
-0.2501971897283317
Arsonist Sets NYC Vegetarian Restaurant on Fire: Police #NewYork - http://t.co/

571it [00:00, 908.56it/s]

...

14.701799278513867
Rly tragedy in MP: Some live to recount horror: ÛÏWhen I saw coaches of my train plunging into water I c... http://t.co/AWWg8kNMZg Over.
15.981278724160894
Rly tragedy in MP: Some live to recount horror: ÛÏWhen I saw coaches of my train plunging into water I called ... http://t.co/HR4GNyGSiC
14.008652097953922
@LauraE303B @SheilaGunnReid A war we'll never win? There r 2500 Yezidi women trapped as slaves.We know where they are. Doing nothing is evil
0.5026196006497088
Think of it as a reality TV show I tell myself to quell the panic rising in my throat. Introvert: Trapped in Tokyo. The 150-square-foot
1.9269332216522574
Been signed off work for a week as i have a trapped nerve and can not move.! Life is a struggle right now.! ????
1.9262019866015931
Confused Westie Believes Dogs Are Trapped In The Computer
http://t.co/aWZoNRYc9y http://t.co/HwJ7GwAVvB
-0.12130495484229664
(#TeambrianMundial) Hollywood Movie About Trapped Miners Released in Chile: 'The 33' Holly... h

3234it [00:03, 824.62it/s]

0.3334565927340343
My new sounds: War Zone https://t.co/hNXRfqRk3P on #SoundCloud
-0.9245325893245964
Sammy is here in this war zone. Jamal spoke to me on the phone. Now my wife is next to speak to me ...what else can&gt; http://t.co/2CppfprxoG
0.12851557930723345
@thelovatoagent omg i feel like i am in a war zone
0.3037953423769828
I think it's time for a cup of tea and a scone before I tackle the war zone that is is my bedroom. I wonder how many condom packets Ill find
0.023981320592729727
Saipan looks like a war zone though
3.4660459962961765
T Shirts $10 male or female get wit me. 2 days until its game changin time. War Zone single will beÛ_ https://t.co/Z0poYR096J
1.238308550220757
Um OK '@dandre_5: Sundays during football seasonfrom about 9 am - 11 pm women shouldn't even log onshit be a complete war zone'
-1.7995071365393693
Just trying to find my my peace in a Zone of war
0.33841442582468595
Twitter is going to be a war zone today  https://t.co/1QHNqCZvod
0.2805611569875923
Th

3263it [00:03, 896.42it/s]


MEG issues Hazardous Weather Outlook (HWO) http://t.co/3X6RBQJHn3
4.09808843302343
#CityofCalgary has activated its Municipal Emergency Plan. #yycstorm
3.72167801881311


In [28]:
sum(test.target == 1)

2125

In [30]:
sum(test.target == 0)

1138

In [31]:
test

id keyword location  \
0         0     NaN      NaN   
1         2     NaN      NaN   
2         3     NaN      NaN   
3         9     NaN      NaN   
4        11     NaN      NaN   
...     ...     ...      ...   
3258  10861     NaN      NaN   
3259  10865     NaN      NaN   
3260  10868     NaN      NaN   
3261  10874     NaN      NaN   
3262  10875     NaN      NaN   

                                                   text  target  
0                    Just happened a terrible car crash       1  
1     Heard about #earthquake is different cities, s...       1  
2     there is a forest fire at spot pond, geese are...       1  
3              Apocalypse lighting. #Spokane #wildfires       1  
4         Typhoon Soudelor kills 28 in China and Taiwan       1  
...                                                 ...     ...  
3258  EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...       1  
3259  Storm in RI worse than last hurricane. My city...       1  
3260  Green Line derailment in Chicago http://t.co/U...       1  
3261  MEG issues Hazardous Weather Outlook (HWO) htt...       1  
3262  #CityofCalgary has activated its Municipal Eme...       1  

[3263 rows x 5 columns]

In [32]:
test.to_csv('submission_naive_bayes1.csv', columns=['id', 'target'], index=False)